# Programación para a Data Science 

## Unidad 6: Preprocesamiento de datos en Python 

## Introducción

A continuación encontraréis una posible solución a los ejercicios propuestos en la PEC. 
Tened en cuenta que pueden existir otras maneras de resolver los ejercicios, que también son válidas 
(y que también permiten obtener la máxima puntuación del ejercicio). 

# Ejercicio 1

Importa el fichero `siri.csv` de la carpeta de datos. En ella encontrarás notas (en un rango de 0 a 10) correspondientes a estudiantes de tres escuelas diferentes para cuatro asignaturas. Una vez importados los datos, realiza las siguientes operaciones:

a) Muestra por pantalla una lista de las diferentes escuelas en los datos. En caso no observar tres escuelas por errores tipográficos, mayúsculas y otros, corrige la nomenclatura para homogeneizar una columna con las tres escuelas. 


**(1 punto)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>



In [1]:
import numpy as np
import pandas as pd

d = pd.read_csv("data/siri.csv", delimiter = ',')

In [2]:
# Vemos con qué variables se codifican las tres escuelas
# Nos damos cuenta que hay diferencias entre mayúsculas/minúsculas
#    así como algún typo  en algunas de ellas

print(d["school"].unique())

['Summer' 'summer' 'Sumer' 'sumer' 'ummer' 'winter' 'Winter' 'spring'
 'Spring' 'Sprng']


In [3]:
# Vamos a corregir la codificación con el mismo nombre de escuela
#    para cada una de las tres

d.loc[(d.school == "Summer") | 
      (d.school == "summer") | 
      (d.school == "Sumer")| 
      (d.school == "sumer")| 
      (d.school == "ummer"), "school"] = "Summer"
d.loc[(d.school == "winter") | 
      (d.school == "Winter") , "school"] = "Winter"
d.loc[(d.school == "spring") | 
      (d.school == "Spring")| 
      (d.school == "Sprng") , "school"] = "Spring"

In [4]:
print(d["school"].unique())

['Summer' 'Winter' 'Spring']



b) Buscar notas claramente anómalas. En caso de encontrar notas anómalas sustitúyelas por NaN.


**(1 punto)**  <span style="font-family: Courier New; background-color: #f2ae72; color: #000000; padding: 3px; ">EI</span> 


In [5]:
#   Vemos que hay notas por encima de 10

d.describe()

,maths,literature,physics,history
count,148.000000,144.000000,145.000000,148.000000
mean,6.125676,3.064583,4.461379,1.193243
std,3.752716,0.440354,8.290475,0.764489
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.400000,1.300000
75%,6.400000,3.325000,5.100000,1.800000
max,50.400000,4.400000,101.400000,2.500000


In [6]:
# Alternativamente lo podemos ver aplicando las funciones max
#    y min al data frame
d.agg([max,min])

,maths,literature,physics,history,school
max,50.4,4.4,101.4,2.5,Winter
min,4.3,2.0,1.0,0.1,Spring


In [7]:
# Observamos también que tenemos valores perdidos
#  antes de corregir las notas erróneas
np.any(d.isna())

True

In [8]:
# Primero controlamos las notas erróneas
# Para corregir las notas erróneas os damos dos 
# soluciones, las dos usando el método applymap
# applymap aplica una función a cada elemento del
# data frame

# La primera es usa una función lambda
#  lambda define una función en línea, dentro del argumento

d.iloc[:,:-1] = d.iloc[:,:-1].applymap(
    lambda x: x if x<=10 else np.nan)

In [9]:
# Alternativamente podemos definir una función fuera
# y aplicarla a cada elemento con la ayuda de appymap

def check_nan(x):
    if x <= 10:
        return x
    else:
        return np.nan
d.iloc[:,:-1] = d.iloc[:,:-1].applymap(check_nan)
    

In [10]:
#Miramos la proporción (en %) de NaNs por variable
d.isna().agg(np.mean)*100

maths         2.000000
literature    4.000000
physics       4.000000
history       1.333333
school        0.000000
dtype: float64


c) Encuentra los valores perdidos. Imputa los valores perdidos por la media de los datos por columna 


**(0.5 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>



In [33]:
# Ya hemos visto en los notebooks de teoría que tenemos el método SimpleImputer del módulo
#    impute de sklearn
# Creamos una instáncia del imputador y luego la aplicamos a nuestros datos
# También podríamos usar el método .fit_transform en vez de hacer esto en 
#   dos pasos 


from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, 
                    strategy='mean')

imp.fit(d.iloc[:,:-1])
d.iloc[:,:-1] = imp.transform(d.iloc[:,:-1])

d) Comprueba que ya no hay valores perdidos en el objeto pandas. 

**(0.5 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>


In [34]:
# Observamos si ahora tenemos valores perdidos
np.any(d.isna())

False

## Ejercicio 2


En este ejercicio trabajaremos un conjunto de datos que se derivó del censo estadounidense de 1990, utilizando una fila por censo
dentro de un grupo de viviendas o bloques. Un grupo de bloques es la unidad geográfica más pequeña para la que La Oficina del Censo de  EE.UU.
publica datos de muestra (un grupo de bloques generalmente tiene una población
de 600 a 3,000 personas).


  - Pace, R. Kelley and Ronald Barry, Sparse Spatial Autoregressions,
      Statistics and Probability Letters, 33 (1997) 291-297





a) Generar un objeto `pandas` de nombre `housings` con los datos que genera la función `datasets.fetch_california_housing()` dentro de `sklearn`. El objeto debe contener todas las variables del conjunto de datos incluida una columna con los valores de  `Prices` (target). 


**(1 punto)**  <span style="font-family: Courier New; background-color: #ffcc5c; color: #000000; padding: 3px; ">EG</span>

In [35]:
# Vemos por la documentación que el método fetch_california_housing
#  del módulo de datasets baja estos datos de internet y nos devuelve 
#  un bunch, de forma similar a iris.
# Vemos que los precios "Prices" están en la variable de target del bunch


import numpy as np
import pandas as pd
from sklearn import datasets

housings_bunch = datasets.fetch_california_housing()

housings = pd.DataFrame(housings_bunch['data'],
                 columns=housings_bunch['feature_names'])
housings['Prices'] =  housings_bunch['target']


b) Obtener, los nombres de las variables, el número total de filas y columnas, y comprobar la posible existencia de valores perdidos en los datasets. Obtener la media, desviación típica, valores máximos y mínimos de todas las variables del data frame.


**(0.5 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>

In [36]:
# El método .describe() nos da la información estadística
print(housings.shape)
print(housings.columns)

# el método .any() nos indica si alguno es True 
print(housings.isna().any(axis=0))
housings.describe()

(20640, 9)
Index(['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup',
       'Latitude', 'Longitude', 'Prices'],
      dtype='object')
MedInc        False
HouseAge      False
AveRooms      False
AveBedrms     False
Population    False
AveOccup      False
Latitude      False
Longitude     False
Prices        False
dtype: bool


,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Prices
count,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,3.870671,28.639486,5.429000,1.096675,1425.476744,3.070655,35.631861,-119.569704,2.068558
std,1.899822,12.585558,2.474173,0.473911,1132.462122,10.386050,2.135952,2.003532,1.153956
min,0.499900,1.000000,0.846154,0.333333,3.000000,0.692308,32.540000,-124.350000,0.149990
25%,2.563400,18.000000,4.440716,1.006079,787.000000,2.429741,33.930000,-121.800000,1.196000
50%,3.534800,29.000000,5.229129,1.048780,1166.000000,2.818116,34.260000,-118.490000,1.797000
75%,4.743250,37.000000,6.052381,1.099526,1725.000000,3.282261,37.710000,-118.010000,2.647250
max,15.000100,52.000000,141.909091,34.066667,35682.000000,1243.333333,41.950000,-114.310000,5.000010


c) Calcula la correlación de la variable `Prices` con el resto de variables. Ordena el vector de correlaciones de mayor a menor en valor absoluto. ¿Qué variable aparece con más correlación con los precios? Interpreta los resultados.


**(0.5 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>

In [37]:
# Calculamos la correlación con el método .corr()

housings_c = housings.corr()
housings_c

,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,Prices
MedInc,1.000000,-0.119034,0.326895,-0.062040,0.004834,0.018766,-0.079809,-0.015176,0.688075
HouseAge,-0.119034,1.000000,-0.153277,-0.077747,-0.296244,0.013191,0.011173,-0.108197,0.105623
AveRooms,0.326895,-0.153277,1.000000,0.847621,-0.072213,-0.004852,0.106389,-0.027540,0.151948
AveBedrms,-0.062040,-0.077747,0.847621,1.000000,-0.066197,-0.006181,0.069721,0.013344,-0.046701
Population,0.004834,-0.296244,-0.072213,-0.066197,1.000000,0.069863,-0.108785,0.099773,-0.024650
AveOccup,0.018766,0.013191,-0.004852,-0.006181,0.069863,1.000000,0.002366,0.002476,-0.023737
Latitude,-0.079809,0.011173,0.106389,0.069721,-0.108785,0.002366,1.000000,-0.924664,-0.144160
Longitude,-0.015176,-0.108197,-0.027540,0.013344,0.099773,0.002476,-0.924664,1.000000,-0.045967
Prices,0.688075,0.105623,0.151948,-0.046701,-0.024650,-0.023737,-0.144160,-0.045967,1.000000


In [38]:
# De la matriz de correlación, nos interesa solo la correlación con la
# variable Prices, así que: 

p = housings_c.Prices
p =  abs(p)
p.sort_values(ascending=False)

# Parece que los precios parecen más relacionados con el sueldo 
# número de habitaciones y latitud que con la población, longitud y 
# ocupación media


Prices        1.000000
MedInc        0.688075
AveRooms      0.151948
Latitude      0.144160
HouseAge      0.105623
AveBedrms     0.046701
Longitude     0.045967
Population    0.024650
AveOccup      0.023737
Name: Prices, dtype: float64

## Ejercicio 3

Con la mismo dataset anterior:




a) Con cada una de las variables `Latitude` y `Longitude`, crea dos variables discretizadas con 10 segmentos, de forma que:

- en las primeras cada intervalo contenga el mísmo número de muestras (equidensidad por segmento) en (`qLat`, `qLon`) 
- en las segundas cada intervalo tengan la misma longitud de intervalo (equidistancia por segmento) en (`sLat`,`sLon`) 


**(1 punto)**  <span style="font-family: Courier New; background-color: #f2ae72; color: #000000; padding: 3px; ">EI</span>

In [39]:

# el método qcut() calcula los cortes mantieniendo 
#    la densidad por segmento

housings["qLat"] = pd.qcut(housings["Latitude"], 10)
housings["qLon"] = pd.qcut(housings["Longitude"], 10)


# El método .cut() obtiene segmentos equiespaciados

housings["sLat"] = pd.cut(housings["Latitude"], 10)
housings["sLon"] = pd.cut(housings["Longitude"], 10)

b) Observa la distribución en las variables discretizadas calculando el número de muestras dentro de cada intérvalo. Podéis mirar el método de pandas .value_counts() para contar valores de cada grupo. 

**(0.5 puntos)** <span style="font-family: Courier New; background-color: #ffcc5c; color: #000000; padding: 3px; ">EG</span> 

In [40]:
# El método .value_counts() es un método propio a pandas, 
#   así que lo podemos aplicar a las variables de interés 

housings['qLat'].value_counts()

(33.63, 33.87]     2160
(34.0, 34.1]       2138
(37.48, 37.81]     2112
(32.539, 33.63]    2092
(36.66, 37.48]     2073
(38.48, 41.95]     2061
(33.87, 34.0]      2058
(37.81, 38.48]     2005
(34.26, 36.66]     2000
(34.1, 34.26]      1941
Name: qLat, dtype: int64

In [41]:
housings['sLat'].value_counts()

(33.481, 34.422]    8987
(37.245, 38.186]    4712
(32.531, 33.481]    1796
(38.186, 39.127]    1740
(36.304, 37.245]    1292
(34.422, 35.363]     747
(39.127, 40.068]     490
(35.363, 36.304]     474
(40.068, 41.009]     324
(41.009, 41.95]       78
Name: sLat, dtype: int64

c) Muestra

- los precios promedio por cada segmento de Latitud (sLat)

- los precios promedio por cada segmento de Longitud (sLon)


**(1 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>

In [42]:
# Agrupamos las Latitudes con el método .groupby()
#  luego calculamos la estadística

housings.groupby('sLat').agg([np.mean,np.std]).Prices

<ipython-input-42-619b3924a459>:4: FutureWarning: ['qLat', 'qLon', 'sLon'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  housings.groupby('sLat').agg([np.mean,np.std]).Prices


,mean,std
sLat,,
"(32.531, 33.481]",1.933825,1.002090
"(33.481, 34.422]",2.287872,1.125460
"(34.422, 35.363]",1.713885,1.081505
"(35.363, 36.304]",0.913435,0.548389
"(36.304, 37.245]",1.570614,1.106603
"(37.245, 38.186]",2.424769,1.224680
"(38.186, 39.127]",1.516993,0.666301
"(39.127, 40.068]",1.048045,0.420524
"(40.068, 41.009]",0.872176,0.258943


In [43]:
# Igualmente para las longitudes 

housings.groupby('sLon').agg([np.mean,np.std]).Prices

<ipython-input-43-0257024d7d8c>:3: FutureWarning: ['qLat', 'qLon', 'sLat'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  housings.groupby('sLon').agg([np.mean,np.std]).Prices


,mean,std
sLon,,
"(-124.36, -123.346]",0.966016,0.359620
"(-123.346, -122.342]",2.640850,1.280013
"(-122.342, -121.338]",2.189859,1.157531
"(-121.338, -120.334]",1.370753,0.595856
"(-120.334, -119.33]",1.317344,1.049821
"(-119.33, -118.326]",2.633149,1.369861
"(-118.326, -117.322]",2.094742,0.969399
"(-117.322, -116.318]",1.740253,0.934753
"(-116.318, -115.314]",0.819301,0.421321


d) Calcula y muestra la media y la desviación típica para todas las variables para cada combinación de Latitud y Longitud en ambos casos (equidensidad por segmento, equidistancia por segmento). 
Observación: quita las filas que contengan algún NaN del dataframe resultante. 

**(1 puntos)**  <span style="font-family: Courier New; background-color: #82b74b; color: #000000; padding: 2px; ">NM</span>

In [22]:
# De forma parecida al ejercio anterior, pero ahora indicando las dos
#    variables en vez de una sola:

qM = housings.groupby(['qLat','qLon']).agg([np.mean, np.std])
qM = qM[qM.notna().all(axis=1)]

<ipython-input-22-454696765839>:4: FutureWarning: ['sLat', 'sLon'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  qM = housings.groupby(['qLat','qLon']).agg([np.mean, np.std])


In [23]:
qM

MedInc             HouseAge             \
                                         mean       std       mean        std   
qLat            qLon                                                            
(32.539, 33.63] (-118.49, -118.3]    2.744420  0.444180  42.400000  13.164346   
                (-118.13, -117.89]   5.882535  2.362263  30.557692   8.564435   
                (-117.89, -117.25]   5.039886  2.312767  18.984252  10.274715   
                (-117.25, -114.31]   3.596781  1.637609  23.563851  11.482094   
(33.63, 33.87]  (-118.49, -118.3]    5.469732  2.003316  30.471545   8.771312   
                (-118.3, -118.13]    3.403173  1.212555  34.924812  10.009260   
                (-118.13, -117.89]   4.494720  1.490364  26.284382   8.139149   
                (-117.89, -117.25]   5.110573  2.142979  19.108932  10.166362   
                (-117.25, -114.31]   3.076170  1.322040  15.777778   7.764367   
(33.87, 34.0]   (-119.91, -118.49]   3.750171  2.090202  38.000000  11.328430   
                (-118.49, -118.3]    4.016103  1.601272  36.275797   9.433230   
                (-118.3, -118.13]    2.485158  0.947699  36.111773   7.248532   
                (-118.13, -117.89]   4.396109  1.483095  31.173824   8.076254   
                (-117.89, -117.25]   4.325288  1.809000  23.413194  11.308754   
                (-117.25, -114.31]   2.945431  1.255333  17.142857  10.069736   
(34.0, 34.1]    (-119.91, -118.49]   7.593832  3.684406  33.323077  10.628834   
                (-118.49, -118.3]    3.937410  2.753832  38.338485  10.339056   
                (-118.3, -118.13]    2.444141  0.989802  38.064516   9.832804   
                (-118.13, -117.89]   3.575224  1.012486  32.117347   6.912793   
                (-117.89, -117.25]   3.447544  1.358013  25.809524  12.313406   
                (-117.25, -114.31]   3.861184  1.805478  22.551724  12.084738   
(34.1, 34.26]   (-119.91, -118.49]   5.148549  1.985111  25.082988   8.939236   
                (-118.49, -118.3]    4.080295  1.777639  32.691849   9.880686   
                (-118.3, -118.13]    4.283538  2.202202  37.904255  11.429384   
                (-118.13, -117.89]   4.865886  2.427514  37.328889   9.403092   
                (-117.89, -117.25]   3.874902  1.739377  26.263941  12.121626   
                (-117.25, -114.31]   3.231963  1.510721  18.988372   8.134884   
(34.26, 36.66]  (-121.98, -121.37]   3.981567  1.810332  30.827586  10.870203   
                (-121.37, -119.91]   3.307965  1.185521  20.988201   9.769224   
                (-119.91, -118.49]   3.353408  1.700751  26.359319  10.934166   
                (-118.49, -118.3]    3.745757  1.401547  26.264463  10.384097   
                (-118.3, -118.13]    3.843348  1.344373  20.855072  11.590152   
                (-118.13, -117.89]   3.504453  1.022524  16.075472  12.359750   
                (-117.89, -117.25]   3.342948  1.198867  14.336364   9.428901   
                (-117.25, -114.31]   2.908234  1.172693  19.736842  10.567079   
(36.66, 37.48]  (-124.351, -122.29]  6.820343  2.355830  31.428571  15.479633   
                (-122.29, -121.98]   5.790986  2.532643  30.359606   9.564897   
                (-121.98, -121.37]   4.589755  1.699340  25.220681  11.340371   
                (-121.37, -119.91]   2.655521  0.961472  24.375000  10.545032   
                (-119.91, -118.49]   3.091592  1.515667  24.951456  13.175683   
                (-118.49, -118.3]    2.889590  1.048031  25.800000   7.299924   
(37.48, 37.81]  (-124.351, -122.29]  4.224052  1.857259  43.025611  11.071680   
                (-122.29, -121.98]   3.783957  1.545953  35.334254  12.673968   
                (-121.98, -121.37]   5.412914  2.082735  21.352941  11.876789   
                (-121.37, -119.91]   3.016095  1.184719  25.194118  11.904634   
                (-119.91, -118.49]   3.447611  0.639684  16.944444   5.915804   
(37.81, 38.48]  (-124.351, -122.29]  4.056617  1.694273  30.296131  12

## Ejercicio 4

### Estandarización de variables.

Importa el dataset `iris` de `sklearn`. Almacena este dataset como un objeto `pandas`, con los correpondientres nombres de variables. 

Llamamos tipificar una variable al proceso de convertirla en una Normal Estándar $N(\mu=0,\sigma =1)$.
Busca en `sklearn.preprocessing` un método que te permita tipificar cada variable numérica del objeto pandas de `iris` de forma que:

- El valor medio de cada columna sea 0
- La desviación típica de cada columna sea 1

a) Aplica el método para crear un nuevo dataframne `iris_typ` que cumpla esas condiciones.  Comprueba que la tipificación es correcta.

**(1 puntos)** <span style="font-family: Courier New; background-color: #ffcc5c; color: #000000; padding: 3px; ">EG</span>

In [24]:
from sklearn import datasets
import pandas as pd
iris_bunch = datasets.load_iris()
iris_pandas = pd.DataFrame(iris_bunch['data'],columns=iris_bunch['feature_names'])
iris_pandas['Species'] = iris_bunch['target_names'][iris_bunch['target']]



In [25]:
# Si miramos la documentación de sklearn.preprocessing, 
#   tenemos el método StandardScaler que hace lo indicado 
#   en el enunciado, así que lo usamos de la forma que nos 
#   indica la documentación:

from sklearn.preprocessing import StandardScaler

iris_typ = iris_pandas.copy()
s = StandardScaler()
s.fit(iris_pandas.iloc[:,:-1])



StandardScaler()

In [26]:
# Usamos el método .fit_transform() para encontrar los valores
#    tipificados 

iris_typ.iloc[:,:-1] = s.fit_transform(iris_typ.iloc[:,:-1])
iris_typ.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,1.500000e+02,1.500000e+02,1.500000e+02,1.500000e+02
mean,-4.736952e-16,-7.815970e-16,-4.263256e-16,-4.736952e-16
std,1.003350e+00,1.003350e+00,1.003350e+00,1.003350e+00
min,-1.870024e+00,-2.433947e+00,-1.567576e+00,-1.447076e+00
25%,-9.006812e-01,-5.923730e-01,-1.226552e+00,-1.183812e+00
50%,-5.250608e-02,-1.319795e-01,3.364776e-01,1.325097e-01
75%,6.745011e-01,5.586108e-01,7.627583e-01,7.906707e-01
max,2.492019e+00,3.090775e+00,1.785832e+00,1.712096e+00


b) Busca y aplica un segundo mecanismo de normalización de forma que:

- el mínimo de la variable sea 0, y 
- el máximo de cada variable sea 1 

Comprueba que la normalizacion es correcta

**(0.5 puntos)** <span style="font-family: Courier New; background-color: #ffcc5c; color: #000000; padding: 3px; ">EG</span>

In [27]:
# En la documentación también vemos el método MinMaxScaler,
#    que hace jústamente lo requerido por el enunciado
# Su aplicación es idéntica al caso anterior 

from sklearn.preprocessing import MinMaxScaler

smm = MinMaxScaler()
iris_typ.iloc[:,:-1] = smm.fit_transform(iris_typ.iloc[:,:-1])
iris_typ.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
count,150.000000,150.000000,150.000000,150.000000
mean,0.428704,0.440556,0.467458,0.458056
std,0.230018,0.181611,0.299203,0.317599
min,0.000000,0.000000,0.000000,0.000000
25%,0.222222,0.333333,0.101695,0.083333
50%,0.416667,0.416667,0.567797,0.500000
75%,0.583333,0.541667,0.694915,0.708333
max,1.000000,1.000000,1.000000,1.000000


### Ejercicio Opcional


La Oficina de Estadísticas de Transporte del Departamento de Transporte de los Estados Unidos (DOT) rastrea el rendimiento en término de puntualidad de los vuelos en EE.UU. operados por grandes compañías aéreas. 

La información resumida sobre el número de vuelos puntuales, retrasados, cancelados y desviados se publica en el Informe mensual del consumidor de viajes aéreos del DOT y en este conjunto de datos de retrasos y cancelaciones de vuelos de 2015. Encontrarás una muestra de estos datos en el fichero `vuelos.csv` en la carpeta de datos. 

Importa el fichero de vuelos. Imputa los valores perdidos con la mediana de cada columna.

Calcula y muestra la matriz de correlación de las siguientes variables a partir de la matriz imputada. 

- `ARRIVAL_DELAY`
- `DIVERTED`
- `CANCELLED`	
- `AIR_SYSTEM_DELAY`
- `SECURITY_DELAY`
- `AIRLINE_DELAY`
- `LATE_AIRCRAFT_DELAY`
- `WEATHER_DELAY`

Interpreta el resultado


 <span style="font-family: Courier New; background-color: #f2ae72; color: #000000; padding: 3px; ">EI</span>

In [28]:
data = pd.read_csv("data/vuelos.csv", 
                   sep=";",
                   decimal=",")
data.head()
f = data[['ARRIVAL_DELAY',
    'DIVERTED',
    'CANCELLED',
    'AIR_SYSTEM_DELAY',
    'SECURITY_DELAY',
    'AIRLINE_DELAY',
    'LATE_AIRCRAFT_DELAY',
    'WEATHER_DELAY']]


In [29]:
from sklearn.impute import SimpleImputer

I = SimpleImputer(strategy="median")
impf= pd.DataFrame(I.fit_transform(f), 
                   columns = f.columns)

In [30]:
impf.head()

,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,-22.0,0.0,0.0,11.0,0.0,7.5,0.0,0.0
1,-9.0,0.0,0.0,11.0,0.0,7.5,0.0,0.0
2,5.0,0.0,0.0,11.0,0.0,7.5,0.0,0.0
3,-9.0,0.0,0.0,11.0,0.0,7.5,0.0,0.0
4,-21.0,0.0,0.0,11.0,0.0,7.5,0.0,0.0


In [31]:
impf.corr()

,ARRIVAL_DELAY,DIVERTED,CANCELLED,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
ARRIVAL_DELAY,1.000000,-0.005171,-0.049490,0.063397,0.172322,0.577452,0.342010,0.489017
DIVERTED,-0.005171,1.000000,-0.009581,-0.001497,-0.001001,-0.003774,-0.001340,-0.003668
CANCELLED,-0.049490,-0.009581,1.000000,-0.014327,-0.009581,-0.036120,-0.012823,-0.035105
AIR_SYSTEM_DELAY,0.063397,-0.001497,-0.014327,1.000000,-0.060304,-0.172941,-0.080708,-0.107038
SECURITY_DELAY,0.172322,-0.001001,-0.009581,-0.060304,1.000000,0.205878,-0.001340,-0.003668
AIRLINE_DELAY,0.577452,-0.003774,-0.036120,-0.172941,0.205878,1.000000,-0.024187,-0.067420
LATE_AIRCRAFT_DELAY,0.342010,-0.001340,-0.012823,-0.080708,-0.001340,-0.024187,1.000000,-0.004909
WEATHER_DELAY,0.489017,-0.003668,-0.035105,-0.107038,-0.003668,-0.067420,-0.004909,1.000000


In [32]:
# Vemos que la variable que tiene más correlación con los retrasos 
#   son la AIRLINE_DELAY, seguida de la WEATHER_DELAY
# La que menos correlación muestra es la de AIR_SYSTEM_DELAY 